# Detectron2 

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">




# Install detectron2

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.5 in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision==0.6 in /usr/local/lib/python3.6/dist-packages (0.6.0+cu101)
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-o1azxu_m
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-o1azxu_m
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266449 sha256=5998a9d61622f5ed9b7f539910826531ee62bc8fa8a948246bbf1c683f58cff6
  Stored in directory: /tmp/pip-ephem-wheel-cache-1i9d3jhw/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2
1.5.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 201

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html


fatal: destination path 'detectron2_repo' already exists and is not an empty directory.
Obtaining file:///content/detectron2_repo
Processing /root/.cache/pip/wheels/68/a5/e7/56401832f23d0b2db351c5b682e466cb4841960b086da65e4e/pycocotools-2.0.2-cp36-cp36m-linux_x86_64.whl
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0
  Found existing installation: detectron2 0.1.3+cu101
    Uninstalling detectron2-0.1.3+cu101:
      Successfully uninstalled detectron2-0.1.3+cu101
  Running setup.py develop for detectron2
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
  Using cached https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/detectron2-0.1.3%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: detectron2 0.3
    Can't uninstall 'detectron2'. No files were found to uninstall.


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Train on a plate dataset

Option 1 : run !wget https://www.dropbox.com/s/rjra44iyzoh19te/plate.zip?dl=0 -O plate.zip

Option 2 : Download the plate.zip from https://drive.google.com/file/d/14wXRkpow5vlVc2ROhHbWelEjwCJCx9b5/view?usp=sharing and upload it


Unzip plate.zip.


In [ ]:
!wget https://www.dropbox.com/s/rjra44iyzoh19te/plate.zip?dl=0 -O plate.zip

!unzip plate.zip 

--2020-11-10 07:10:15--  https://www.dropbox.com/s/rjra44iyzoh19te/plate.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rjra44iyzoh19te/plate.zip [following]
--2020-11-10 07:10:15--  https://www.dropbox.com/s/raw/rjra44iyzoh19te/plate.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1b7a11801d8ee9cb335c3a67d0.dl.dropboxusercontent.com/cd/0/inline/BC7aPGduvPluOmrcKsXrRMBE_G1v4feAEIgZnQ_ArVpXHJq2Z0k0xRbz-5SLiCjrHeY7Gb_sjD946gjaDQV22ReVhkqfwOgGx2xZ153v-q65guMwKuTbdELqU6IFyrwDIfU/file# [following]
--2020-11-10 07:10:15--  https://uc1b7a11801d8ee9cb335c3a67d0.dl.dropboxusercontent.com/cd/0/inline/BC7aPGduvPluOmrcKsXrRMBE_G1v4feAEIgZnQ_ArVpXHJq2Z0k0xRbz-5SLiCjrHeY7Gb_sjD946gjaDQV22ReVhkqfwOgGx2xZ153v-q65guMwKuT

In [ ]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools
import cv2

# write a function that loads the dataset into detectron2's standard format
def get_plate_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json")  
    with open(json_file) as f:
        imgs_anns = json.load(f)
    dataset_dicts = []
    filename_list = []
    bbox_list = []
    bbox_num = []
    for index,(a, v) in enumerate(imgs_anns.items()):
        if index == 0:
            filename_list = v
        elif index == 2:
            bbox_list = v   
    for i in range(len(filename_list)):
        count = 0
        for j in range(len(bbox_list)):
            if str(filename_list[i]["id"]) == str(bbox_list[j]['image_id']):
                count += 1
        bbox_num.append(count)
    k = 0
    for i in range(len(filename_list)):
        record = {}       
        filename = os.path.join(img_dir, filename_list[i]["file_name"])
        height, width = cv2.imread(filename).shape[:2]
        record["file_name"] = filename
        record["image_id"] = i
        record["height"] = 1080
        record["width"] = 1920
        objs = []
        for j in range(k,k+bbox_num[i]):            
            if bbox_list[j]["image_id"] != record["image_id"]:
                assert False
            bbox = []
            bbox = [int(bbox_list[j]["bbox"][0]),int(bbox_list[j]["bbox"][1]),
                    int(bbox_list[j]["bbox"][0])+int(bbox_list[j]["bbox"][2]),
                    int(bbox_list[j]["bbox"][1])+int(bbox_list[j]["bbox"][3])]         
            obj = {
                "bbox":bbox,
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation":bbox_list[j]["segmentation"],
                "category_id":0,
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        k += bbox_num[i]
        dataset_dicts.append(record)
    return dataset_dicts
from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "val"]:
    DatasetCatalog.register("plate_" + d, lambda d=d: get_plate_dicts("plate/" + d))
    MetadataCatalog.get("plate_" + d).set(thing_classes=["plate"])
plate_metadata = MetadataCatalog.get("plate_train")  

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



In [ ]:
import random

dataset_dicts = get_plate_dicts("plate/train")

for d in random.sample(dataset_dicts, 3):

    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plate_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

Now, let's fine-tune a coco-pretrained R50-FPN Mask R-CNN model on the plate dataset. It takes ~12 minutes to train 2000 iterations on Colab's K80 GPU.


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("plate_train",)
cfg.DATASETS.TEST = ()  
cfg.DATALOADER.NUM_WORKERS = 5
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2000   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[11/10 07:13:57 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[11/10 07:14:07 d2.engine.train_loop]: Starting training from iteration 0
[11/10 07:14:34 d2.utils.events]:  eta: 0:44:09  iter: 19  total_loss: 1.740  loss_cls: 0.832  loss_box_reg: 0.001  loss_mask: 0.690  loss_rpn_cls: 0.197  loss_rpn_loc: 0.011  time: 1.3270  data_time: 0.0505  lr: 0.000005  max_mem: 2669M
[11/10 07:15:01 d2.utils.events]:  eta: 0:44:28  iter: 39  total_loss: 1.629  loss_cls: 0.742  loss_box_reg: 0.001  loss_mask: 0.658  loss_rpn_cls: 0.215  loss_rpn_loc: 0.015  time: 1.3524  data_time: 0.0064  lr: 0.000010  max_mem: 2669M
[11/10 07:15:29 d2.utils.events]:  eta: 0:44:18  iter: 59  total_loss: 1.429  loss_cls: 0.590  loss_box_reg: 0.001  loss_mask: 0.593  loss_rpn_cls: 0.207  loss_rpn_loc: 0.016  time: 1.3606  data_time: 0.0064  lr: 0.000015  max_mem: 2669M
[11/10 07:15:56 d2.utils.events]:  eta: 0:43:33  iter: 79  total_loss: 1.133  loss_cls: 0.423  loss_box_reg: 0.001  loss_mask: 0.523  loss_rpn_cls: 0.153  loss_rpn_loc: 0.012  time: 1.3537  data_time: 0.0062  lr:

Now, we perform inference with the trained model on the plate validation dataset. First, let's create a predictor using the model we just trained:



In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("plate_val", )
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_plate_dicts("plate/val")
for d in random.sample(dataset_dicts, 10):   
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=plate_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )

    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.